In [6]:
# Stylesheet
plt.style.use('seaborn-white')

plt.rcParams['font.sans-serif'] = "Tahoma"
plt.rcParams['font.family'] = "sans-serif"
plt.rcParams['font.size'] = 14
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['axes.labelweight'] = 'normal'
plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14
plt.rcParams['legend.fontsize'] = 14
plt.rcParams['figure.titlesize'] = 14
plt.rcParams['lines.linewidth'] = 3
params = {"ytick.color" : "#535956",
          "xtick.color" : "#535956",
          "axes.labelcolor" : "#535956",
          "axes.edgecolor" : "#535956",
          "text.color": "#535956"}
plt.rcParams.update(params)

### Get distance_from_others data from combined_replicates/

In [5]:
# Imports
import json
import numpy as np
import plotly.plotly as py
import matplotlib as mpl
# mpl.use('Agg')
# %matplotlib inline
from matplotlib import cycler
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import os
import glob2
import re

In [3]:
# Try on this data of 2 Replicates
with open("distance_bw_workers/Q0.15_W0.5_D1.0_T0.5_wb3.json", "r") as f:
    data = json.load(f)

In [4]:
len(data), len(data[0]), len(data[0][0]["distances"])

(17, 16320, 51)

In [7]:
# data[0][0]

### Get distances to 49 others for 1 bee over 160 timesteps

In [7]:
def get_data_1_bee(data, replicate_i, bee_start_i):
    # Loop over 50 lists in 1 timestep
    # Each list is 1 bee and its distance to 49 other bees at this timestep
    # List of 160 lists, each list has 50 values
    one_bee_160_t = []
    
    # Loop over 160 timesteps here
    # number_of_distances = len(data[0]["Replicate 1"]["distance_from_others"])
    number_of_distances = len(data[0])
    
    for bee in range(bee_start_i, number_of_distances, 51):  # for other bees: every 101?
        # dist_to_49_others = data[replicate_i]["Replicate {}".format(replicate_i+1)]["distance_from_others"][bee]["distances"]
        dist_to_49_others = data[replicate_i][bee]["distances"]
        one_bee_160_t.append(dist_to_49_others)

    return one_bee_160_t


In [8]:
# Test function
# print(len(one_bee_160_t))
# print(one_bee_160_t)
bee_over_time = get_data_1_bee(data, 0, 0)
# print(len(bee_over_time))
# bee_over_time[0]

### Threshold nearest neighbors at max = 2 distance units 

In [9]:
def get_avg_neighbor_distances(data_list):
    # Loop over each timestep list
    # List of 160 floats for 1 bee's average distance from neighbors over time
    nearest_neighbors = []

    for t_list in data_list:
        # 160 avg values for distance to nearest neighbors
        # Threshold nearest neighbors at max = 2 distance units
        avg_list = []
        for val in t_list:
            if val <= 2:
                avg_list.append(val)
                # print(val)
        nearest_neighbors.append(np.mean(avg_list))

    return nearest_neighbors

In [10]:
# # Test
# print(len(get_avg_neighbor_distances(bee_over_time)))
# get_avg_neighbor_distances(bee_over_time)

### Use get_data_1_bee & get_avg_neighbor_distances to get 50 lists for 50 bees -- in 1 replicate  -- later average over all 10 replicates

In [11]:
def get_distances_50_bees(data, replicate_i):
    # List of 50 lists 
    neighbor_distances_all_bees = [] 

    for i in range(51):
        # print(i)
        one_bee = get_data_1_bee(data, replicate_i, i)
        avg_distances = get_avg_neighbor_distances(one_bee)
        neighbor_distances_all_bees.append(avg_distances)
        
    return neighbor_distances_all_bees

In [12]:
# Test
# get_distances_50_bees(data, 1)

In [13]:
def all_bees_all_time(data, replicate_i):
    ''' For 1 replicate swarm, get list of all bees avg distance
    from nearest neighbors over all 160 timesteps '''

    # Avg over all 50 bees for each of 160 timesteps
    # Get a list of 160 values
    all_distances = get_distances_50_bees(data, replicate_i)
    
    all_bees_over_time = []
    for i in range(320):
        # For each timestep, i, get values of each of the 50 bees at that i 
        one_t = [item[i] for item in all_distances]
        # Get the avg of of all bees at the timestep
        avg_all_bees = np.mean(one_t)
        all_bees_over_time.append(avg_all_bees)
        
    return all_bees_over_time

In [14]:
# Test
# all_bees_all_time(data, 1)

### Average over all replicates in JSON

In [15]:
# # Get all replicates' lists into a list
# all_replicates = []
# for i in range(len(data)):
#     one_replicate_data = all_bees_all_time(data, i)
#     all_replicates.append(one_replicate_data)

In [16]:
# # Store means of 10 replicates
# all_reps_avgs = []

# # Also get min and max for range in plotting 
# mins = []; maxes = []
# #
# for i in range(160):
#     one_timestep = [item[i] for item in all_replicates]
#     # Get the avg of of all bees at the timestep
#     avg_all_reps_all_bees = np.mean(one_timestep)
#     min_val = min(one_timestep)
#     max_val = max(one_timestep)
    
#     all_reps_avgs.append(avg_all_reps_all_bees)
#     mins.append(min_val)
#     maxes.append(max_val)


In [17]:
def avg_all_replicates(data_json):
    # Get all replicates' lists into a list
    all_replicates = []
    for i in range(len(data_json)):
        one_replicate_data = all_bees_all_time(data_json, i)
        all_replicates.append(one_replicate_data)
        
    # Store means of 10 replicates
    all_reps_avgs = []

    # Also get min and max for range in plotting 
    mins = np.min(all_replicates, axis=0)
    maxs = np.max(all_replicates, axis=0)
    
    data = {
        "all" : all_replicates,
        "min" : mins,
        "max" : maxs
    }

    return data

In [18]:
# with open("combined_replicates/Q0.15_W0.01_D0.35_T0.5_wb1.json", "r") as f:
#     data = json.load(f)

- Colorbrewer: http://colorbrewer2.org/#type=sequential&scheme=YlGnBu&n=5
- tsplot: https://seaborn.pydata.org/generated/seaborn.tsplot.html

In [19]:
def plot_me(json_name):
    
    with open("distance_bw_workers/{}".format(json_name), "r") as f:
        json_data = json.load(f)
    
    bee_data = avg_all_replicates(json_data)
    
    
    # sns.set_context("poster", rc={"font.size":8,"axes.titlesize":8,"axes.labelsize":5})
    # sns.set_context("poster", rc={'text.color': "#2eaf53"})

    
    fig, ax = plt.subplots(1, figsize=(8, 5))
    sns.set(font_scale = 1.5)
    sns.set_style("whitegrid", {'axes.grid' : False, 'text.color': '#535956', 
                                'axes.labelcolor': '#535956', 
                                'xtick.color': '#535956',
                                'ytick.color': '#535956',
                                'axes.linewidth': 1.5,
                               'axes.edgecolor': '#535956'})
    
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)

    # sns.set_style("whitegrid")
    
    # Setup minmax bounds
    ax.plot(bee_data["min"], color='#2c7fb8', alpha=0.2)
    ax.plot(bee_data["max"], color='#2c7fb8', alpha=0.2)
    
    # Fill between minmax bounds
    ax.fill_between([i for i in range(len(bee_data["min"]))], bee_data["min"], bee_data["max"], 
                    where = bee_data["max"] >= bee_data["min"], facecolor='#2c7fb8', alpha=0.15, interpolate=True)

    # Average with stds 68 and 95
    g = sns.tsplot(data=bee_data["all"], estimator=np.mean, color="#2eaf53", alpha=0.8, ci=[68, 95], ax=ax, err_kws={"color" : "#a1dab4", "alpha" : 0.})
    g.lines[2].set_linewidth(3)
    g.set(ylim=(0,2))
    
    processed_fname = json_name[:-5]
    
    ax.set(xlabel='Time (step)', ylabel='Distance')
    plt.title("Distance between workers over time \n ({})".format(processed_fname))
    plt.savefig('/Users/dieumynguyen/Desktop/Projects/bee_communication/figures/distance_bw_workers/{}.pdf'.format(processed_fname))
    
    plt.close()

In [20]:
# plot_me("Q0.15_W0.01_D0.35_T0.5_wb1.json")
# plt.show()

In [23]:
# Iterate through all (256) JSON's and produce 1 figure each
reps_list = list(map(lambda x : x.split("/")[-1], glob2.glob("distance_bw_workers/*.json")))

for i, r in enumerate(reps_list):
    print("{}. Plotting distance bw workers for: {}".format(i+1, r))
    plot_me(r)

1. Plotting distance bw workers for: Q0.15_W0.0005_D0.005_T0.005_wb3.json


/Users/dieumynguyen/anaconda3/lib/python3.6/site-packages/seaborn/timeseries.py:183: UserWarning:

The tsplot function is deprecated and will be removed or replaced (in a substantially altered version) in a future release.



2. Plotting distance bw workers for: Q0.15_W0.0005_D0.005_T0.5_wb3.json
3. Plotting distance bw workers for: Q0.15_W0.0005_D0.01_T0.005_wb3.json
4. Plotting distance bw workers for: Q0.15_W0.0005_D0.01_T0.5_wb3.json
5. Plotting distance bw workers for: Q0.15_W0.0005_D0.05_T0.005_wb3.json
6. Plotting distance bw workers for: Q0.15_W0.0005_D0.05_T0.5_wb3.json
7. Plotting distance bw workers for: Q0.15_W0.0005_D0.1_T0.005_wb3.json
8. Plotting distance bw workers for: Q0.15_W0.0005_D0.1_T0.5_wb3.json
9. Plotting distance bw workers for: Q0.15_W0.0005_D0.5_T0.005_wb3.json
10. Plotting distance bw workers for: Q0.15_W0.0005_D0.5_T0.5_wb3.json
11. Plotting distance bw workers for: Q0.15_W0.0005_D1.0_T0.005_wb3.json
12. Plotting distance bw workers for: Q0.15_W0.0005_D1.0_T0.5_wb3.json
13. Plotting distance bw workers for: Q0.15_W0.001_D0.005_T0.005_wb3.json
14. Plotting distance bw workers for: Q0.15_W0.001_D0.005_T0.5_wb3.json
15. Plotting distance bw workers for: Q0.15_W0.001_D0.01_T0.005_w